In [1]:
import pandas as pd
#import torch
#import nltk
import numpy as np
import random
#import tensorflow as tf

#### Domain features/objects/relevant functions

In [3]:
class Domain1:
    def __init__(self, parameters):
        None
    
class Domain2:
    def __init__(self, parameters):
        None
    
    
def find_approximate_occurance_based_on_structure(): #computes similarity between two sentences based on word structure and word contribution
    '''Example:
    s1: It is very overpriced and not very tasty
    s2: This lacks the features and is very expensive.
    s3: This lacks the features and is very expensive for a model A laptop.
    s4: It is very overpriced and not very tasty for a Michelin star restaraunt. 
    overpriced= expensive
    tasty = feaatures
    very = very
    not = lack
    Michelin star = model A 
    
    Domain dependent words--> get_domain_dependent_words()
    Domain independent words --> get_independent_words()
    
    'very expensive' is together/combined word 
    '''
    None
    
def find_approximate_occurance_based_on_words():
    '''Example:
    
    '''
    None
    
def find_inter_domain_similarity():
    None

#### Approach:

1. Baseline simple method: BiLSTM + CRF.  This is a baseline model. We need a model with high coverage (high false positives) and then trim the selection using the below model. This can either be simple rules, syntactic structures, just any model with high number of false positives. 


2. Given a sentence S, train an encoder-decoder network to reproduce the sentence. Compute last hidden state representation based on model M (S-> S). After training for a while, for each sentence: for each aspect word: remove BA, IA labelled words, and get representation. 2nd model takes this representation and the original sentence + some noisy cases and outputs a 1 or 0 depending on task. Effectively, 2nd model is learning to use incomplete representation and other info to learn whether it is an aspect/opinion term. 


To find out word contribution, by training a model based on missing words. So, if a word is removed, get some representation. Embed the ones with aspects missing to a similar space. Meaning if I have a sentence: I love their ice-cream so much. Then remove ice-cream, get a representation of the sentence, and a representation of the normal (with ice-cream sentence). Using these representations, map as similar (1) in a Siamese net. Now take all other (except opinion terms) and use same model to get representation. But the final model outputs the representations as dissimilar.

    G(F(I love their so much), F(I love their ice cream so much)) = 1
    G(F(love their ice cream so much), F(I love their ice cream so much)) = 0
    G(F(I love their ice cream), F(I love their ice cream so much)) = 2
    G(F(I love ice cream so much), F(Sentence)) = 



        Another approach could be if we remove the word then from all the mined words, if we replace the term with it, does the sentence still make sense (is it legible)-> then it is an aspect word. 

        So given s= I love their ice cream so much, if we remove ice-cream then replace it with say 50 previous aspect words like -> football, passing, nachos, ambience, does it still make sense? A model M is trained to identify whether a sentence still makes sense. 


#### What is an aspect and what is an opinion term?

S1: 'Plus it is small and reasonably light so I can take it with me to and from work. -> small, reasonably light

S2:'If internet connectivity is important I would recommend going with a dell net book for 50 bucks more, or buy a USB wireless card.' -> recommend, and internet connectivity;USB wireless card

S3: "Rao is a good restaurant, but it's nothing special." -> good, nothing special

S4: iLife is easily compatible with Microsoft Office so you can send and receive files from a PC.--> easily compatible-->  iLife;Microsoft Office 

S5: "people are rude bit again it's new york!" -> rude; people

S6: "The speed is incredible and I am more than satisfied." --> incredible, 'more than satisfied', 'speed'





***
## Method 1: Using missing words 

1. Remove respective word and train a model to either detect whether sentence is legible or not. 2nd way: can we get a frequency method to fill in the blank with respective HISTORIC/TRAINING 'other' words, 'opinion' words or 'aspect' words and see which make the most sense.

2. Remove respective word(s) and first train an autoencoding-decoding network (normal sentences- no missing words). Then train a 2nd model to take normal repreenation and missing representation, and output 'OP' , 'AP' or 'OTH' 
    1. Would adding noise to the first autoencoding network be beneficial? No:Cause representations need to be diff.
    2. Should the 1st model be frozen when we begin training the 2nd model?
    3. Should a new sentence be autencoded few times and then use pretrained 2nd model from training?
    4. Is using a dependency structure as in Wenya's better. THere's something here, a better way to capture missing word importance representation- something like attention in this dependency structure
    5. How do we get prospective words to choose words to remove?: <b>Principle: All nouns are equal, but some nouns are more important. What makes these more important? </b>
    6. How do we resolve multi aspect/opinion (BA,IA) expressions? 
    7. Should we use a MISSING token to represent absent words and obtain representation? 
    8. Which step of the LSTM/Structure should we use (last hidden state vs missing state vs Tree RNN structure state)
    

#### Answering the question: Why are some nouns (of the same tag) more relevant- as an aspect/opinion- than others?  Below we see examples of such occurances (same word tag- but one of them is an aspect/opinion and the other an 'other' )

In [8]:
import spacy
import nltk
import pandas as pd
import numpy as np
import pickle, csv
import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd
import string
torch.manual_seed(10)
from nltk import Tree

#nlp = spacy.load('en') #load spacy model

### Exploring POS tags and where multiple nouns/types occur: How are they different

In [9]:
with open("Final_data/laptop_lower_additional_training_list.pickle") as p1:
    laptop_data = pickle.load(p1)
    

In [11]:
def get_more_than_n_tag_occurs(tagged_tokens, n= 2 , tag_list= ["NOUN","PRON"], tag_index = 1):
    filtered_toks = [i for i in tagged_tokens if i[tag_index] in tag_list]
    if(len(filtered_toks)>=n):
        return filtered_toks
    else:
        return len(filtered_toks)
    

#rest_data = pd.read_csv("Final_data/Semeval_14_ver1/Combined_restaurant.csv")

' '.join(laptop_data[0][0]), laptop_data[0][1], laptop_data[0][2]



#' '.join(laptop_data[0][0]), get_more_than_n_tag_occurs(laptop_data[0][2])
#How is night and cord and battery_life different? 
#From a human level, 'night' is representing when the person ('I') is doing something. 
#Whereas cord

#good is the only adjective here
#laptop_data[0][0], get_more_than_n_tag_occurs(laptop_data[0][2])

('i charge it at night and skip taking the cord with me because of the good battery life',
 [5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 5, 5, 3, 1, 2],
 [('i', 'PRON', 'PRP', 'nsubj'),
  ('charge', 'VERB', 'VBP', 'ROOT'),
  ('it', 'PRON', 'PRP', 'dobj'),
  ('at', 'ADP', 'IN', 'prep'),
  ('night', 'NOUN', 'NN', 'pobj'),
  ('and', 'CCONJ', 'CC', 'cc'),
  ('skip', 'VERB', 'VB', 'conj'),
  ('taking', 'VERB', 'VBG', 'xcomp'),
  ('the', 'DET', 'DT', 'det'),
  ('cord', 'NOUN', 'NN', 'dobj'),
  ('with', 'ADP', 'IN', 'prep'),
  ('me', 'PRON', 'PRP', 'pobj'),
  ('because', 'ADP', 'IN', 'prep'),
  ('of', 'ADP', 'IN', 'pcomp'),
  ('the', 'DET', 'DT', 'det'),
  ('good', 'ADJ', 'JJ', 'amod'),
  ('battery', 'NOUN', 'NN', 'compound'),
  ('life', 'NOUN', 'NN', 'pobj')])

In [12]:
print("NOUN case:", ' '.join(laptop_data[5][0]), get_more_than_n_tag_occurs(laptop_data[5][2], tag_list=["NOUN","PROPN"]))
print("ADJ case:", ' '.join(laptop_data[5][0]), get_more_than_n_tag_occurs(laptop_data[5][2], tag_list=["ADJ"]))

'''
Very interesting example
quality is actually an Opinion term<- how do we know this, because it's part of an ADJ modification (high quality, good quality, etc,)What if it occured independently?
Ex: It's quality is suspect. <- THis is an aspect 

So many cases of ADJ and NOUN yet only few are chosen. Why: answer this

Also, opinions do not have to be restricted to aspects 
'''

list(zip(laptop_data[5][0],map(lambda x: id_to_tag[x], laptop_data[5][1])))


('NOUN case:', 'it is of high quality has a killer gui is extremely stable is highly expandable is bundled with lots of very good applications is easy to use and is absolutely gorgeous', [('quality', 'NOUN', 'NN', 'pobj'), ('killer', 'NOUN', 'NN', 'compound'), ('gui', 'NOUN', 'NN', 'nsubj'), ('lots', 'NOUN', 'NNS', 'pobj'), ('applications', 'NOUN', 'NNS', 'pobj')])
('ADJ case:', 'it is of high quality has a killer gui is extremely stable is highly expandable is bundled with lots of very good applications is easy to use and is absolutely gorgeous', [('high', 'ADJ', 'JJ', 'amod'), ('stable', 'ADJ', 'JJ', 'acomp'), ('expandable', 'ADJ', 'JJ', 'acomp'), ('good', 'ADJ', 'JJ', 'amod'), ('easy', 'ADJ', 'JJ', 'acomp'), ('gorgeous', 'ADJ', 'JJ', 'acomp')])


NameError: global name 'id_to_tag' is not defined

In [18]:
#Make autoencoder model, train it to reproduce same sentence by computing final hidden representation. 
#Ans the question is adding noise to this worth it? 

#Noise reconstruction means we try to get representations through incomplete info to be approximately the same as with complete info.
#If we are using a model to learn how the difference between a noisy representation and a compelete representation (missing word vs no missing word), then we want the representation to be different, and not almost the same. 

#2nd question why does a missing word which has the same tag/context not be included

***
## How do parser trees change when important words go missing

In [20]:
#Get dependency parser tree: https://stackoverflow.com/questions/36610179/how-to-get-the-dependency-tree-with-spacy


def tok_format(tok):
    return "_".join([tok.orth_, tok.tag_])



def to_nltk_tree(node, token_tagged = False):
    if node.n_lefts + node.n_rights > 0:
        if token_tagged:
            return Tree(tok_format(node), [to_nltk_tree(child, True) for child in node.children])
        else:
            return Tree(node.orth_, [to_nltk_tree(child) for child in node.children])
    else:
        if token_tagged:
            return tok_format(node)
        else:
            return node.orth_

def get_dependency_parser_tree(seq):
    """Input is a list of tokens"""
    doc = nlp(unicode(' '.join(seq)))
    return [to_nltk_tree(sent.root, True).pretty_print() for sent in doc.sents]

#now by removing some aspects/opinions we observe the dependency trees
get_ignore_indices = lambda filter_list, input_tag_seq: [i for i, tag in enumerate(input_tag_seq) if tag in filter_list]
#w1 w3 w4
def get_filtered_sentence(seq, tag_seq, filter_list= [2]):
    ignore_indices = get_ignore_indices(filter_list, tag_seq)
    #print(ignore_indices)
    return [word for i, word in enumerate(seq) if i not in ignore_indices]

#' '.join(get_filtered_sentence(laptop_data[0][0], laptop_data[0][1], [1,2])) #removing BA and IA words (1 and 2)


In [21]:
print ' '.join(laptop_data[0][0])
get_dependency_parser_tree(laptop_data[0][0])

I charge it at night and skip taking the cord with me because of the good battery life
                             charge_VBP                                                                
   ______________________________|____________________________                                          
  |     |      |       |                                   skip_VB                                     
  |     |      |       |                     _________________|_______________                          
  |     |      |       |                taking_VBG                        because_IN                   
  |     |      |       |          __________|_________         _______________|_________                
  |     |      |     at_IN    cord_NN              with_IN    |                      life_NN           
  |     |      |       |         |                    |       |       __________________|________       
I_PRP it_PRP and_CC night_NN   the_DT               me_PRP  of_IN  the_DT    

[None]

In [22]:
filtered_sentence = get_filtered_sentence(laptop_data[0][0], laptop_data[0][1], [1,2]) #removing BA and IA words (1 and 2)
print filtered_sentence
get_dependency_parser_tree(filtered_sentence)

['I', 'charge', 'it', 'at', 'night', 'and', 'skip', 'taking', 'the', 'with', 'me', 'because', 'of', 'the', 'good']
                    charge_VBP                                                           
   _____________________|___________________________________                              
  |     |      |        |                                skip_VB                         
  |     |      |        |                  _________________|______________               
  |     |      |        |             taking_VBG                       because_IN        
  |     |      |        |         ________|_________                _______|_________     
  |     |      |      at_IN      |               with_IN           |              good_JJ
  |     |      |        |        |                  |              |                 |    
I_PRP it_PRP and_CC  night_NN  the_DT             me_PRP         of_IN             the_DT



[None]

In [23]:
filtered_sentence = get_filtered_sentence(laptop_data[0][0], laptop_data[0][1], [3,4]) #removing BO and IO words (1 and 2)
print filtered_sentence
get_dependency_parser_tree(filtered_sentence)

['I', 'charge', 'it', 'at', 'night', 'and', 'skip', 'taking', 'the', 'cord', 'with', 'me', 'because', 'of', 'the', 'battery', 'life']
                             charge_VBP                                                         
   ______________________________|____________________________                                   
  |     |      |       |                                   skip_VB                              
  |     |      |       |                     _________________|________                          
  |     |      |       |                taking_VBG                 because_IN                   
  |     |      |       |          __________|_________         ________|_________                
  |     |      |     at_IN    cord_NN              with_IN    |               life_NN           
  |     |      |       |         |                    |       |         _________|________       
I_PRP it_PRP and_CC night_NN   the_DT               me_PRP  of_IN    the_DT           

[None]

In [256]:
filtered_sentence = laptop_data[0][0]
#filtered_sentence.insert(5,'night')
#filtered_sentence.remove('night')
get_dependency_parser_tree(filtered_sentence)

                              i_PRP                                                                 
   _____________________________|__________________________                                          
  |      |     |                                        skip_NN                                     
  |      |     |        ___________________________________|_______________                          
  |      |     |       |             taking_VBG                        because_IN                   
  |      |     |       |         ________|_________         _______________|_________                
  |      |     |       |     cord_NN            with_IN    |                      life_NN           
  |      |     |       |        |                  |       |       __________________|________       
it_PRP at_IN and_CC night_NN  the_DT             me_PRP  of_IN  the_DT            good_JJ battery_NN



[None]

In [251]:
filtered_sentence.remove('night')
get_dependency_parser_tree(filtered_sentence)

                          charge_VBP                                                         
   ___________________________|____________________________                                   
  |     |      |     |                                  skip_VB                              
  |     |      |     |                    _________________|________                          
  |     |      |     |               taking_VBG                 because_IN                   
  |     |      |     |         __________|_________         ________|_________                
  |     |      |     |     cord_NN              with_IN    |               life_NN           
  |     |      |     |        |                    |       |         _________|________       
I_PRP it_PRP at_IN and_CC   the_DT               me_PRP  of_IN    the_DT           battery_NN



[None]

***
### In doing the experiment with using missing word representations, we can have many variations. 
1. Do we train the model to reproduce complete sentence? (As stated adding noise seems counterintuitive)
2. If we use seq2seq model encoders, we could do either of the three:
    1. Remove the word completely and use last hidden representation as the decider for type of missing word
    2. Replace the word with an 'UNK' token and compute immediate representation at that step, compared to previous steps. 
    3. Combine 1 and 2 both 
    
3. Let's say for a given input sentence: w1 w2 w3... wn, how do we:
    1. Decide candidates to remove?: Need another model for this?? Or do we use a simple heuristic: all nouns, adjs and verbs
    2. What if multiple BA IA word seq is there? Do we remove them together?
    3. When we remove, do we use the standard seq2seq architectures, or do we use the Tree RNN type?
    4. Given that we have detected w1 to w3 as BA..IA then how do we include this information when W6 has been detected as an opinion word. Do we get representations for all possible words at different time steps, and use their config as well in deciding the final seq labels ?
    
    
#### In doing experiment with missing word FOR using historic label representative data--> do we try each characteristic word (so Hello, hi, hey--> binned into 1 cluster, similarly OP_WORDS that are similar binned into 1 cluster) and use a MODEL to detect which resulting sentence makes most sense. 

In [ ]:
#A simple implementation of the first model 
#Steps

#1) Take a simple heuristic rule to take positive examples and negative examples for each category (BA, BA.IA, BO, BO.IO, Ot)
# 2nd model is a simple siames



*** 
### Script to get tag-token frequencies

In [261]:
'''Script to get frequencies of POS tags for opinion and aspect words'''
from nltk import word_tokenize, pos_tag
from nltk.data import load
#tagdict = load('help/tagsets/upenn_tagset.pickle')

examples_of_aspect_as_ADJ = []
examples_of_opinion_as_NOUN = []


def get_freq_pos_opinion_and_aspect(df, limit =-1):
    '''Input is a dataframe'''
    #bopinion_tags = {tag:0 for tag in tagdict.keys()}
    #baspect_tags = {tag:0 for tag in tagdict.keys()}
    #iopinion_tags = {tag:0 for tag in tagdict.keys()}
    #iaspect_tags = {tag:0 for tag in tagdict.keys()}
    bopinion_tags = {} 
    baspect_tags = {}
    iopinion_tags = {} 
    iaspect_tags = {}
    '''
    if(limit==-1):
        limit = len(df)
        print_it = False
    else:
        limit = limit #for debugging
        '''
    for sentence, opinion, aspect in zip(df.Sentence, df.Opinions, df.Aspects):
        '''
        if(limit<=0):
            break
        limit-=1
        '''
        '''NLTK
        #text = word_tokenize(sentence) #<-NLTK
        #pos_tagged_text = pos_tag(text) #<-NLTK
        '''
        text = nlp(unicode(sentence))
        pos_tagged_text = [(str(token), str(token.pos_)) for token in text]
        
        
        for seqs in opinion.split(';'): #find all opinion terms-> can be done together with aspect terms but cleaner this way
            for i, opinion_term in enumerate(seqs.split()):
                for token_pos_tuple in pos_tagged_text:
                    token, pos_tag = token_pos_tuple[0], token_pos_tuple[1]
                    if(opinion_term == token):
                        if(i>1):#IO
                            if(pos_tag not in iopinion_tags):
                                iopinion_tags[pos_tag] = 1
                            else:
                                iopinion_tags[pos_tag] += 1
                        else:#BO
                            if(pos_tag not in bopinion_tags):
                                bopinion_tags[pos_tag] = 1
                            else:
                                bopinion_tags[pos_tag] += 1
                        '''    
                        if(limit> 0 and print_it):
                            print(sentence, token_pos_tuple)
                        '''
                        if(pos_tag == "NOUN"):
                            examples_of_opinion_as_NOUN.append((sentence, token_pos_tuple))
                        
        for seqs in aspect.split(';'): #find all aspect terms
            for i, aspect_term in enumerate(seqs.split()):
                for token_pos_tuple in pos_tagged_text:
                    token, pos_tag = token_pos_tuple[0], token_pos_tuple[1]
                    if(aspect_term == token):
                        if(i>1):#IA
                            if(pos_tag not in iaspect_tags):
                                iaspect_tags[pos_tag] = 1
                            else:
                                iaspect_tags[pos_tag] += 1
                        else:#BA
                            if(pos_tag not in baspect_tags):
                                baspect_tags[pos_tag] = 1
                            else:
                                baspect_tags[pos_tag] += 1
                       
                        if(pos_tag == 'ADJ'):
                            examples_of_aspect_as_ADJ.append((sentence, token_pos_tuple))
    return bopinion_tags, iopinion_tags, baspect_tags, iaspect_tags

#laptop_df = pd.read_csv("./Final_data/Semeval_14_ver1/Combined_laptop.csv")
#laptop_freq_dicts = get_freq_pos_opinion_and_aspect(laptop_df)

#rest_df = pd.read_csv("./Final_data/Semeval_14_ver1/Combined_restaurant.csv")
#rest_freq_dicts = get_freq_pos_opinion_and_aspect(rest_df)

In [108]:
laptop_freq_dicts, rest_freq_dicts

(({'ADJ': 1507,
   'ADP': 49,
   'ADV': 372,
   'CCONJ': 3,
   'DET': 36,
   'INTJ': 1,
   'NOUN': 256,
   'NUM': 4,
   'PART': 26,
   'PROPN': 1,
   'VERB': 477},
  {'ADJ': 28,
   'ADP': 12,
   'ADV': 23,
   'CCONJ': 2,
   'DET': 13,
   'NOUN': 18,
   'PART': 6,
   'PRON': 1,
   'VERB': 16},
  {'ADJ': 183,
   'ADP': 36,
   'ADV': 1,
   'CCONJ': 5,
   'DET': 11,
   'NOUN': 2395,
   'NUM': 68,
   'PART': 26,
   'PRON': 4,
   'PROPN': 464,
   'VERB': 262},
  {'ADJ': 7,
   'ADP': 15,
   'ADV': 1,
   'CCONJ': 1,
   'DET': 25,
   'NOUN': 196,
   'NUM': 4,
   'PART': 1,
   'PROPN': 37,
   'VERB': 6}),
 ({'ADJ': 2665,
   'ADP': 39,
   'ADV': 195,
   'CCONJ': 1,
   'DET': 3,
   'INTJ': 2,
   'NOUN': 137,
   'NUM': 2,
   'PART': 9,
   'PROPN': 2,
   'VERB': 418},
  {'ADJ': 1, 'DET': 6, 'NOUN': 9},
  {'ADJ': 288,
   'ADP': 110,
   'ADV': 5,
   'CCONJ': 20,
   'DET': 12,
   'NOUN': 3933,
   'NUM': 3,
   'PART': 3,
   'PRON': 1,
   'PROPN': 257,
   'VERB': 155,
   'X': 3},
  {'ADJ': 37,
   'ADP': 

***
## 1) Training an autoencoder for sentences. (We then freeze the best working model for later use in missing words)

##### All experiments until the next section (missing words) is with regards to training the autoencoder

In [13]:
'''Autoencoder for sentences (Encoder-decoder network)'''
import torch 
import torch.nn as nn 
from torch.autograd import Variable
from torch import optim 
import torch.nn.functional as F
import time
import random
from gensim.models.keyedvectors import KeyedVectors
use_cuda = torch.cuda.is_available() 


In [14]:
'''
class Encoder_BiLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, embedding_dim):
        super(Encoder_BiLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim//2, bidirectional = True) #hidden dim//2 cause it's a bilstm (automatically concats across 1st axis)
        self.hidden = 
        
    def init_hidden(self):
        return (autograd.Variable(torch.randn(2, 1, self.hidden_dim//2)), autograd.Variable(torch.randn(2,1, self.hidden_dim//2)))
    
    def forward(self, input_seq, hidden):
        inputs =
'''

class Encoder_GRU(nn.Module):
    def __init__(self, input_domain_size, hidden_dims, embedding_dims, with_pretrained_embeddings = False,  pretrained_embedding_matrix = None, freeze_embeddings = False, input_domain_word2id = None):
        super(Encoder_GRU, self).__init__()
        self.hidden_dims = hidden_dims
        self.gru = nn.GRU(embedding_dims, hidden_dims)
        
        if(not with_pretrained_embeddings):
            self.embedding = nn.Embedding(input_domain_size, embedding_dims)
        else:
            #embeddings = load_embeddings(path_to_embeddings, input_domain_word2id, embedding_dims) #this is a numpy array
          
            assert input_domain_size == pretrained_embedding_matrix.size()[0]#should match
            assert embedding_dims == pretrained_embedding_matrix.size()[1] 
            self.embedding = nn.Embedding(input_domain_size, embedding_dims)
            self.embedding.weight = nn.Parameter(pretrained_embedding_matrix)
            if(freeze_embeddings):
                self.embedding.requires_grad = False
                
        
    def init_hidden(self):
        '''Get the first hidden state'''
        #hidden_var =  (autograd.Variable(torch.randn(1, 1, self.hidden_dims)))
        hidden_var = autograd.Variable(torch.zeros(1, 1, self.hidden_dims)) #makes more sense to be 0 for all? Why would randomness across different inputs help?
        if(use_cuda):
            return hidden_var.cuda()
        else:
            return hidden_var
        
    def forward(self, input_at_current_time_step, hidden):
        '''Expected input is of shape 1*dims'''
        embedded_input_seq = self.embedding(input_at_current_time_step).view(1,1,-1) #turn into RNN format [seq_length, minibatch_size, dimensions]
        output, hidden = self.gru(embedded_input_seq, hidden)
        return output, hidden

In [15]:
class Decoder_GRU(nn.Module):
    def __init__(self, output_domain_size, hidden_dims, embedding_dims, with_pretrained_embeddings = False,  pretrained_embedding_matrix = None, freeze_embeddings = False, output_domain_word2id = None): 
        #output size in this case remains same as input_size (since we're reproducing in the same domain)
        super(Decoder_GRU, self).__init__()
        self.hidden_dims = hidden_dims
        self.gru = nn.GRU(embedding_dims, hidden_dims)
        self.out = nn.Linear(hidden_dims, output_domain_size)
        self.softmax = nn.LogSoftmax(dim = 1) #dim is 1 (column), since output of linear is of shape batch=1* output_vocab
        
        if(not with_pretrained_embeddings):
            self.embedding = nn.Embedding(output_domain_size, embedding_dims)
        else:
            #embeddings = load_embeddings(path_to_embeddings, output_domain_word2id, embedding_dims) #this is a numpy array
            assert output_domain_size == pretrained_embedding_matrix.size()[0]#should match
            assert embedding_dims == pretrained_embedding_matrix.size()[1] 
            self.embedding = nn.Embedding(output_domain_size, embedding_dims)
            self.embedding.weight = nn.Parameter(pretrained_embedding_matrix)
            if(freeze_embeddings):
                self.embedding.requires_grad = False
                
    def init_hidden(self):
        #hidden_var =  (autograd.Variable(torch.randn(1, 1, self.hidden_dims)))
        hidden_var = autograd.Variable(torch.zeros(1, 1, self.hidden_dims))
        if(use_cuda):
            return hidden_var.cuda()
        else:
            return hidden_var
        
    def forward(self, input_at_time_step, hidden):
        embedded_input = self.embedding(input_at_time_step).view(1,1,-1)
        relud_embedding = F.relu(embedded_input) #just cause it works for others
        output_at_time_step, hidden = self.gru(relud_embedding, hidden) #in the current case where we loop through each time step, the output is the same as the hidden state
        softmaxed_prediction = self.softmax(self.out(output_at_time_step[0])) #index output_time_step by row to get shape 1*hidden_dims
        return softmaxed_prediction, hidden

In [16]:

'''Training model for each sequence function. This is a subsidiary of the training function'''
def train_model(input_seq_variable, output_seq_variable, encoder, decoder, encoder_optimizer, decoder_optimizer, cost_func, start_token, end_token, max_length = 30, teacher_forcing_ratio = 0.4):
    #.....................................................
    #This function trains the encoder and decoder architecture jointly to reproduce input and output sequences
    #High level overview:
    #1) With step by step encoder, get the hidden state at each time step
    #2) Use last step hidden rep, any relevant step (ex:missing variable) hidden rep or attention based cumulation to pass as decoder initial hidden step
    #3) Now decode using hidden info and other ways to include encoding info (IF REQUIRED). Use softmax and top 1 prediction to get output distribution
    #4) Calculate cost (in this case should be negative log likelihood)
    #5) Backpropate through loss
    #6) Update variables based on optimizer
    #......................................................
    
    #1.1) initialize encoder hidden state
    encoder_hidden = encoder.init_hidden()
    
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad() #Reset accumulated gradients from previous batch
    loss = 0
    
    input_length = input_seq_variable.size()[0]
    output_length = output_seq_variable.size()[0]
    
    
    '''We do not for this model need encoder_outputs since attention is not used and we do not make use of historic encoder outputs'''
    #encoder_outputs = autograd.Variable(torch.zeros(max_length, encoder.hidden_dims)) #store encoder hidden states for later use
    #encoder_outputs = encoder_outputs.cuda() if use_cuda else encoder_outputs
    
    
    
    #1.2) Get encoded representations 
    for encoder_time_step in range(input_length):
        encoder_output, encoder_hidden = encoder(input_seq_variable[encoder_time_step], encoder_hidden) 
        #encoder_outputs[encoder_time_step] = encoder_output[0][0] #store encoder hidden state (output in this case since we're looping individually)
        
        
    #2.1) for the decoder, we have to add the start token 
    decoder_input = autograd.Variable(torch.LongTensor([[start_token]]))
    decoder_input = decoder_input.cuda() if use_cuda else decoder_input
    
    #2.2) Initialize hidden state for decoder
    decoder_hidden = encoder_hidden #encoder_outputs[-1]<- this won't be true in case we don't loop so avoid
    
    use_teacher_forcing = True if random.random()< teacher_forcing_ratio else False
    
    #3) Run decoder and optimize to reproduce input
    
    if(use_teacher_forcing):
        for decoder_time_step in range(output_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden) #replace with other variants later
            loss+= cost_func(decoder_output, output_seq_variable[decoder_time_step]) #adding loss for this sequence
            decoder_input = output_seq_variable[decoder_time_step]
    
    else:
        for decoder_time_step in range(output_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            top_prob, top_index = decoder_output.data.topk(1)
            next_input_id = top_index[0][0] #use this index as input from vocab
            
            decoder_input = autograd.Variable(torch.LongTensor([[next_input_id]]))
            decoder_input = decoder_input.cuda() if use_cuda else decoder_input
            
            loss+= cost_func(decoder_output, output_seq_variable[decoder_time_step])
            if(next_input_id == end_token):
                '''should outputting an EOS when it is not be heavily punished by loss function? 
                However, that may lead to future longer sentences (as short sentences are punished).
                Regardless, how can we implement that'''
                break
           
        
    #Optimize variables based on loss
    loss.backward()
    
    encoder_optimizer.step()
    decoder_optimizer.step()
    
    return loss.data[0]/output_length #return average loss across output sequence


def test_model(input_seq_variable, encoder, decoder, cost_func, start_token, end_token, output_seq_variable =None, no_testing = False, max_length = 30):
    #Same same but no weight updates/teacher forcing
    
    #1) Initialize encoder 1st hidden state and reset loss
    
    encoder_hidden = encoder.init_hidden()
    
    loss = 0 
    
    #2) Pass sequence through encoder network
    input_length = input_seq_variable.size()[0]
    
    for encoder_time_step in range(input_length):
        encoder_output, encoder_hidden = encoder(input_seq_variable[encoder_time_step], encoder_hidden)
        
    #3) Initialize decoder output

    decoder_input = autograd.Variable(torch.LongTensor([[start_token]])) #remember dim is 1*1*-1
    decoder_input = decoder_input.cuda() if use_cuda else decoder_input
    decoder_hidden = encoder_hidden 
    
    #4) Repeat decoder output until end_token is reached (for real world application-otherwise for testing stop when eos is reached)
    predicted_sequence = []
    if(no_testing):
        #Reproduce sentence
        while(next_input_id != end_token):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            
            top_prob, top_index = decoder_output.data.topk(1)
            next_input_id = top_index[0][0] #use this index as input from vocab
            
            decoder_input = autograd.Variable(torch.LongTensor([[next_input_id]]))
            decoder_input = decoder_input.cuda() if use_cuda else decoder_input
            predicted_sequence.append(top_index)
        
        return predicted_sequence
           
                
    else:
        #Testing based on output_length
        output_length = output_seq_variable.size()[0]
        for decoder_time_step in range(output_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            
            top_prob, top_index = decoder_output.data.topk(1)
            next_input_id = top_index[0][0] #use this index as input from vocab

            decoder_input = autograd.Variable(torch.LongTensor([[next_input_id]]))
            decoder_input = decoder_input.cuda() if use_cuda else decoder_input
            predicted_sequence.append(top_index)
            
            loss+= cost_func(decoder_output, output_seq_variable[decoder_time_step])
            if(next_input_id == end_token):
                '''should outputting an EOS when it is not be heavily punished by loss function? 
                However, that may lead to future longer sentences (as short sentences are punished).
                Regardless, how can we implement that'''
                break
                
        return loss.data[0]/output_length, predicted_sequence

In [208]:
'''
m = autograd.Variable(torch.randn(3,4))
a = nn.Linear(4,5)
print(a(m).size())
x = F.softmax(a(m), dim =1)
print(x.data.topk(1))
x.data.topk(1)[1][0], x.data.topk(1)[1][0][0]
'''

'\nm = autograd.Variable(torch.randn(3,4))\na = nn.Linear(4,5)\nprint(a(m).size())\nx = F.softmax(a(m), dim =1)\nprint(x.data.topk(1))\nx.data.topk(1)[1][0], x.data.topk(1)[1][0][0]\n'

In [17]:
#Prepare training files: input--> output (where output is the same)




#Load relevant data for training
def get_dataset_info(file_name = "laptop", training_data_additional = False, get_inverse = False, lower = True):
    
    if(lower):
        opt_string = "lower" #hardcoded
    else:
        opt_string = "final"
        
        
    if(training_data_additional):
        training_data = pickle.load(open("Final_data/{}_{}_additional_training_list.pickle".format(file_name, opt_string)))
    else:
        training_data = pickle.load(open("Final_data/{}_{}_normal_training_list.pickle".format(file_name, opt_string)))
        
    tag2id = pickle.load(open("Final_data/{}_{}_tag2id.pickle".format(file_name, opt_string)))
    
    vocab2id = pickle.load(open("Final_data/{}_{}_vocab.pickle".format(file_name, opt_string)))
    
    if("<START>" in vocab2id.keys()): #assuming <START> and <END> always co-occur
        start_token = vocab2id["<START"]
        end_token = vocab2id["<END>"]
    else:
        start_token = vocab2id["<START>"] = len(vocab2id)
        end_token = vocab2id["<END>"] = len(vocab2id)
    
    if(not get_inverse):
        return training_data, tag2id, vocab2id, start_token, end_token
    else:
        id2vocab = {id_:token for token, id_ in vocab2id.items()}
        id2tag = {id_:tag for tag, id_ in tag2id.items()}
        
        return training_data, tag2id, vocab2id, start_token, end_token, id2vocab, id2tag

#Load pretrained embeddings function 
def load_embeddings(path_to_embeddings, word2id, embedding_dim = 25):
    '''
    Input: This only takes in w2v format. So convert all other embedding types/vectors to w2v format: 
    Output: Torch variable with embeddings only belonging to and indexed by word2id
    
    For glove:
    Use python -m gensim.scripts.glove2word2vec -i <GloVe vector file> -o <Word2vec vector file>
    as per https://radimrehurek.com/gensim/scripts/glove2word2vec.html
    
    
    W2V format is:
    <Num vectors> <dimensionality>
    <word1> <vector rep>
    <word2> <vector rep>
    ....and so on
    '''
    word_vectors = KeyedVectors.load_word2vec_format(path_to_embeddings, binary = False)
    
    
    corpus_embeddings = np.zeros((len(word2id), embedding_dim))
    #Should words not found in embedding file be random or zeros? 
    #Currently 0 and then they are trained anyway
    '''Should we use a try/catch block here so that words that are not in vocab force an exception?
    Or should we choose every word in vocab and see if it is in embedding vocab-> worst case |V|*|embedding_vocab|
    Or vice versa, same worst case but early stopping 
    Or use key retrieval so all words in embedding vocab are checked if they belong to dictionary key
    '''
    for word in word2id.keys():
        if word in word_vectors.vocab:
            corpus_embeddings[word2id[word]] = np.array(word_vectors[word])
        
    return torch.from_numpy(corpus_embeddings).float()


'''Training the simple encoder-decoder'''
def form_indexed_sequence_variable(sequence, word_to_id, end_token):
    #Input sequence is a list of word tokens
    #Output sequence is a list of id tokens based on provided word to id mapping
    input_seq = [word_to_id[token] for token in sequence] + [end_token]
    input_seq_var = autograd.Variable(torch.LongTensor(input_seq)).view(-1,1) #make a 2 rank tensor
    if(use_cuda):
        return input_seq_var.cuda()
    else:
        return input_seq_var
    
def generate_training_sequence(training_dataset, word_to_id, end_token):
    '''Generator function to yield training example 
    Not very useful in this case but for large datasets it can save memory if we pass the dataset by reference.
    Use hpy (guppy for py2) to explore memory usage in the heap
    '''
    random.shuffle(training_dataset) 
    for training_example in training_dataset:
        yield(form_indexed_sequence_variable(training_example[0], word_to_id, end_token), form_indexed_sequence_variable(training_example[1], word_to_id, end_token))
    
    
def generate_testing_sequence(testing_dataset, word_to_id, end_token, testing_size = 50):
    '''Generator function to yield testing example 
    Not very useful in this case but for large datasets it can save memory if we pass the dataset by reference.
    Use hpy (guppy for py2) to explore memory usage in the heap
    '''
    random.shuffle(testing_dataset) 
    for training_example in testing_dataset[:testing_size]:
        yield(form_indexed_sequence_variable(training_example[0], word_to_id, end_token), form_indexed_sequence_variable(training_example[1], word_to_id, end_token))
        
def generate_sentence_from_id(seq, id_to_word):
    return map(lambda x: id_to_word[x], seq)

In [18]:
'''Training script'''

'''Training function
Note: We use stochastic optimization (variables are updated after each iteration in a given epoch)
'''    

def trainIters(encoder, decoder, n_epochs, enc_learning_rate, dec_learning_rate, enc_optimizer, dec_optimizer, loss_function, start_token, end_token,  training_data, testing_data, vocab2id, testing_size = 50, save_when_lowest_loss = False):
    global min_testing_loss 
    encoder_optimizer = enc_optimizer(encoder.parameters(), lr = enc_learning_rate)
    decoder_optimizer = dec_optimizer(decoder.parameters(), lr = dec_learning_rate)
    loss_function = loss_function
    #
    for epoch in range(n_epochs):
        print("At epoch: {}".format(epoch))
        t1 = time.time()
        training_gen = generate_training_sequence(training_data, vocab2id, end_token)
        loss = 0
        iters = 0
        for input_seq, target_seq in training_gen:
            loss += train_model(input_seq, target_seq, encoder, decoder, encoder_optimizer, decoder_optimizer, loss_function, start_token, end_token)
            if(iters%500==0):
                tm = time.time()
                print("At iteration {}, time taken: {}".format(iters, tm-t1))
            iters+=1
        print("Training Loss for epoch: {}".format(loss/iters))
        t2 = time.time()
        print("time for epoch: {}".format(t2-t1))
        
        te_loss = 0 
        te_iters = 0
        testing_gen = generate_testing_sequence(testing_data, vocab2id, end_token, testing_size)
        for input_seq, target_seq in testing_gen:
            te_loss_temp, predicted_seq = test_model(input_seq, encoder, decoder, loss_function, start_token, end_token, target_seq)
            te_loss += te_loss_temp
            if(te_iters%100==0):
                tm = time.time()
                print("At iteration {}, time taken: {}".format(te_iters, tm-t1))
            te_iters+=1
        testing_loss = te_loss/te_iters
        print("Testing Loss for epoch: {}".format(testing_loss))
        t2 = time.time()
        
        if(save_when_lowest_loss):
            if(testing_loss < min_testing_loss):
                torch.save(encoder.state_dict(), "Trained_models/Autoencoder_trained_models/glove50dims/best_enc_state.pth")
                torch.save(decoder.state_dict(), "Trained_models/Autoencoder_trained_models/glove50dims/best_dec_state.pth")
                torch.save(encoder, "Trained_models/Autoencoder_trained_models/glove50dims/best_enc.pth")
                torch.save(decoder, "Trained_models/Autoencoder_trained_models/glove50dims/best_dec.pth")
                pickle.dump(testing_loss, open("Trained_models/Autoencoder_trained_models/glove50dims/min_te_loss.p",'wb'))
                min_testing_loss = testing_loss
                print("Saved model for best testing loss")
       # print("time for epoch: {}".format(t2-t1))
        
        
#form_indexed_sequence_variable(laptop_data[0][0], laptop_vocab2id, end_token)

In [19]:
laptop_data, tag2id, laptop_vocab2id, start_token, end_token = get_dataset_info()
vocab_len = len(laptop_vocab2id)
num_total_examples = len(laptop_data)



'''The below is for training an autoencoder model'''
training_first_index = 0
training_last_index = int(num_total_examples*0.7)
testing_first_index = training_last_index + 1

tokenized_sentences_list = map(lambda x: x[0], laptop_data)
dataset_for_autoencoding = [(tokenized_sentence, tokenized_sentence) for tokenized_sentence in tokenized_sentences_list]
training_data = dataset_for_autoencoding[:training_last_index+1] 
testing_data = dataset_for_autoencoding[testing_first_index:] 

assert len(training_data) + len(testing_data)  == len(dataset_for_autoencoding) #Sanity check

In [23]:
        
'''Model and hyperparameters'''

already_exists = True
pretrained_embeddings_needed = True

if(pretrained_embeddings_needed):
    pretrained_embedding_matrix_encoder = load_embeddings("./Final_data/embeddings/glove/glove50dimw2v_format.txt", laptop_vocab2id, 50)
    pretrained_embedding_matrix_decoder = pretrained_embedding_matrix_encoder #Only for this case, since output domain is same
 #model already exists  

encoder_hidden_dims = 50
encoder_embedding_dims = 50
decoder_hidden_dims = 50
decoder_embedding_dims = 50

if(already_exists == True):
    
    encoder_path = "Trained_models/Autoencoder_trained_models/glove50dims/encoder1_2state.pth"
    encoder1 = Encoder_GRU(vocab_len, encoder_hidden_dims, encoder_embedding_dims, True, pretrained_embedding_matrix_encoder)
    encoder1.load_state_dict(torch.load(encoder_path))
    
    decoder_path = "Trained_models/Autoencoder_trained_models/glove50dims/decoder1_2state.pth"
    decoder1 = Decoder_GRU(vocab_len, decoder_hidden_dims, decoder_embedding_dims, True, pretrained_embedding_matrix_decoder)
    decoder1.load_state_dict(torch.load(decoder_path))
    
    min_testing_loss = pickle.load(open("Trained_models/Autoencoder_trained_models/glove50dims/min_te_loss.p",'rb'))
    print("Loaded model with lowest test loss of {}".format(min_testing_loss))
    
    
    
else:
    
    encoder1 = Encoder_GRU(vocab_len, encoder_hidden_dims, encoder_embedding_dims, True, pretrained_embedding_matrix_encoder)
    
    
    decoder1 = Decoder_GRU(vocab_len, decoder_hidden_dims, decoder_embedding_dims, True, pretrained_embedding_matrix_decoder)
    min_testing_loss = 1e3

if(use_cuda):
    encoder1 = encoder1.cuda()
    decoder1 = decoder1.cuda()
    
enc_learning_rate = 0.01
enc_optimizer = optim.SGD
dec_learning_rate = 0.01
dec_optimizer = optim.SGD

loss_function = nn.NLLLoss()




    


Loaded model with lowest test loss of 3.15490326473


In [283]:
#trainIters(encoder1, decoder1, 25, enc_learning_rate, dec_learning_rate, enc_optimizer, dec_optimizer, loss_function, start_token, end_token, training_data, testing_data, laptop_vocab2id, len(testing_data), save_when_lowest_loss = True)


At epoch: 0
At iteration 0, time taken: 0.0375430583954
At iteration 500, time taken: 17.9576630592
At iteration 1000, time taken: 34.0831320286
At iteration 1500, time taken: 51.5019700527
At iteration 2000, time taken: 70.5592730045
Training Loss for epoch: 2.97441833938
time for epoch: 75.095277071
At iteration 0, time taken: 75.10455513
At iteration 100, time taken: 76.083204031
At iteration 200, time taken: 77.0557579994
At iteration 300, time taken: 77.9593000412
At iteration 400, time taken: 78.906648159
At iteration 500, time taken: 79.8624432087
At iteration 600, time taken: 80.9124979973
At iteration 700, time taken: 81.7745850086
At iteration 800, time taken: 82.6545710564
At iteration 900, time taken: 83.6630151272
Testing Loss for epoch: 4.03362233307
At epoch: 1
At iteration 0, time taken: 0.0176389217377
At iteration 500, time taken: 21.3936100006
At iteration 1000, time taken: 47.0536820889
At iteration 1500, time taken: 62.7475450039
At iteration 2000, time taken: 82.5

At iteration 200, time taken: 69.5505051613
At iteration 300, time taken: 70.5095582008
At iteration 400, time taken: 71.4240880013
At iteration 500, time taken: 72.3052170277
At iteration 600, time taken: 73.4944601059
At iteration 700, time taken: 74.4661900997
At iteration 800, time taken: 75.3854341507
At iteration 900, time taken: 76.4065320492
Testing Loss for epoch: 4.23263543267
At epoch: 11
At iteration 0, time taken: 0.0288090705872
At iteration 500, time taken: 15.9946050644
At iteration 1000, time taken: 31.9137830734
At iteration 1500, time taken: 49.1142990589
At iteration 2000, time taken: 64.9301729202
Training Loss for epoch: 2.92504424282
time for epoch: 69.1401059628
At iteration 0, time taken: 69.1517670155
At iteration 100, time taken: 69.9394369125
At iteration 200, time taken: 70.7138791084
At iteration 300, time taken: 71.5077610016
At iteration 400, time taken: 72.3225600719
At iteration 500, time taken: 73.1056659222
At iteration 600, time taken: 73.925647974


At iteration 500, time taken: 16.0615341663
At iteration 1000, time taken: 31.9992961884
At iteration 1500, time taken: 47.7398610115
At iteration 2000, time taken: 64.7002971172
Training Loss for epoch: 2.79825875066
time for epoch: 69.187415123
At iteration 0, time taken: 69.195374012
At iteration 100, time taken: 70.434595108
At iteration 200, time taken: 71.4868819714
At iteration 300, time taken: 72.5125031471
At iteration 400, time taken: 73.4985041618
At iteration 500, time taken: 74.554197073
At iteration 600, time taken: 75.4909260273
At iteration 700, time taken: 76.457321167
At iteration 800, time taken: 77.3724889755
At iteration 900, time taken: 78.3004620075
Testing Loss for epoch: 4.84196761776
At epoch: 22
At iteration 0, time taken: 0.0322380065918
At iteration 500, time taken: 15.8388741016
At iteration 1000, time taken: 32.1600050926
At iteration 1500, time taken: 47.9942359924
At iteration 2000, time taken: 64.5558459759
Training Loss for epoch: 2.80621796711
time f

In [ ]:
#Another question should the autoencoder be trained jointly with future objective function? 
#That would mean that it is encoding across all sentences such that the encoding is useful 
#Is it better to not use just an encoder which is trained jointly with missing words such that the representations obtained are different for each  

***
## 2) Stimulate missing words and HOW TO MAKE AN ARCHITECTURE TO CAPTURE MISSING WORD(S) CONTRIBUTIONS

#### Word interactions is the next step-> How do we figure the best interaction network
#### Tree tracing of words seemed a distant possibility-> it captures the interaction as well as the contribution--> we want to guess where the most important interactions/contributions occur 

In [6]:
laptop_expanded_data = get_dataset_info(training_data_additional=True)[0]

NameError: name 'get_dataset_info' is not defined

In [25]:
laptop_expanded_data[0]

(['i',
  'charge',
  'it',
  'at',
  'night',
  'and',
  'skip',
  'taking',
  'the',
  'cord',
  'with',
  'me',
  'because',
  'of',
  'the',
  'good',
  'battery',
  'life'],
 [5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 5, 5, 3, 1, 2],
 [('i', 'PRON', 'PRP', 'nsubj'),
  ('charge', 'VERB', 'VBP', 'ROOT'),
  ('it', 'PRON', 'PRP', 'dobj'),
  ('at', 'ADP', 'IN', 'prep'),
  ('night', 'NOUN', 'NN', 'pobj'),
  ('and', 'CCONJ', 'CC', 'cc'),
  ('skip', 'VERB', 'VB', 'conj'),
  ('taking', 'VERB', 'VBG', 'xcomp'),
  ('the', 'DET', 'DT', 'det'),
  ('cord', 'NOUN', 'NN', 'dobj'),
  ('with', 'ADP', 'IN', 'prep'),
  ('me', 'PRON', 'PRP', 'pobj'),
  ('because', 'ADP', 'IN', 'prep'),
  ('of', 'ADP', 'IN', 'pcomp'),
  ('the', 'DET', 'DT', 'det'),
  ('good', 'ADJ', 'JJ', 'amod'),
  ('battery', 'NOUN', 'NN', 'compound'),
  ('life', 'NOUN', 'NN', 'pobj')])

In [26]:
def filter_sentence(tokenized_sentence_tuple, remove_by_seq_tag = [tag2id['BA']], remove_by_pos_tag = ['NOUN'], random_ignore =0.8):
    #Can do batch processing? 
    #Random ignore is used when words may not 
    filtered_tokenized_sentence = []
    tokenized_sentence = tokenized_sentence_tuple[0]
    absa_tags = tokenized_sentence_tuple[1]
    pos_tags = tokenized_sentence_tuple[2]
    
    filtered_sentences = [] #this contains the list of output filtered sentences
    sure_skip_indices = []
    soso_skip_indices = []
    random_ignore = [] # randomly drop some non important words from filtered lists->do later
    for index, token in enumerate(tokenized_sentence_tuple):
        #for each token check if 
        if absa_tags[index] in remove_by_seq_tag:
            skip_indices.append([index])
        elif pos_tags[index] in remove_by_pos_tag:
            soso_skip_indices.append([index])
    
    #
    final_indices = sure_skip_indices + random.choice(soso_skip_indices)     # closest_dist(sure_skipsoso_skip_indices)
        
    #Make sure skip sentences and then just do a permutation of nearby words for other candidates-> choose some randomly
    #+ Choose some randomly from sosoindices to remove
    #Label whichever is a 1 
    

laptop_expanded_data[0]

SyntaxError: invalid syntax (<ipython-input-26-829f6fc736b6>, line 9)

In [4]:
random.choice([1,2,3])

TypeError: choice() takes exactly 2 arguments (3 given)

In [5]:
laptop_data[0][0]

NameError: name 'laptop_data' is not defined

In [259]:
###Stimulating missing words
#The ball was quite round and its feeling on kicking was awesome. 
#zip(laptop_data[20][0], laptop_data[5][1])#, ' '.join(map(lambda x: str(x),laptop_data[5][1]))
laptop_data, 

[('it', 5),
 ('is', 5),
 ('of', 5),
 ('high', 3),
 ('quality', 1),
 ('has', 5),
 ('a', 5),
 ('killer', 5),
 ('gui', 5),
 ('is', 5),
 ('extremely', 5),
 ('stable', 3),
 ('is', 5),
 ('highly', 5),
 ('expandable', 3),
 ('is', 5),
 ('bundled', 5),
 ('with', 5),
 ('lots', 5),
 ('of', 5),
 ('very', 5),
 ('good', 3),
 ('applications', 5),
 ('is', 5),
 ('easy', 3),
 ('to', 5),
 ('use', 5),
 ('and', 5),
 ('is', 5),
 ('absolutely', 5),
 ('gorgeous', 3)]

In [15]:
' '.join(laptop_data[2][0]), laptop_data[2][1]

('the tech guy then said the service center does not do 1to1 exchange and i have to direct my concern to the sales team which is the retail shop which i bought my netbook from',
 [5,
  5,
  5,
  5,
  5,
  5,
  1,
  2,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5])

In [ ]:
#Next step: Generate candidates for removal (missing words)

def generate_candidate_missing_words_and_labels():
    '''Takes input sequence: removes aspect and opinion words jo'''
    '''Given a sentence, get dependency parser and tags
    Remove certain words based on rules? or random noise-> 
    Train jointly with encoder?
    
    '''
    None
    
def generate_noisy_input():
    None
    
    
def best_fit_missing_word_rep():
    '''Make hash embeddings for all words--> assign to buckets depending on number
    Learn buckets for all possible variations
    Try to fit each bucket rep into missing phrase and see which fits best 3
    '''
    None
